# Advanced Spatial Analysis: Spatial Accessibility

## Overview
In this lecture, we will investigate a method to measure spatial accessibility. The method is called the two-step floating catchment area (2SFCA). The method has been spotlighted as it includes the interactions among supply, demand, and mobility, and it is straightforward, making this method relatively easy to replicate and reproduce. 
The method consists of two steps, as you can tell from its name. The first step is to calculate the supply-to-demand ratio of each infrastructure (e.g., hospitals). The second step is to sum the ratio from the locations of demand. In both steps, the method considers the interaction between the supply and demand that is only accessible within a particular threshold travel cost (e.g., 30 minutes or 10 km).  

### First step:

$$\huge R_j = \frac{S_j}{\sum_{k\in {\left\{d_{kj} \le d_0 \right\}}}^{}P_k}$$
where<br>
$R_j$: the supply-to-demand ratio of location $j$. <br>
$S_j$: the degree of supply (e.g., number of doctors) at location $j$. <br>
$P_k$: the degree of demand (e.g., population) at location $k$. <br>
$d_{kj}$: the travel cost between locations $k$ and $j$. <br>
$d_0$: the threshold travel cost of the analysis. 

### Second step:
$$\huge A_i = \sum_{j\in {\left\{d_{ij} \le d_0 \right\}}} R_j$$
where<br>
$A_i$: the accessibility measures at location $i$. <br>
$R_j$: the supply-to-demand ratio of location $j$. <br>

![](./data/fig1.jpg)


Source: <br>
* Luo, W., & Wang, F. (2003). Measures of spatial accessibility to health care in a GIS environment: Synthesis and a case study in the Chicago region. *Environment and Planning B: Planning and Design*, 30(6), 865–884. <br>
* Data is obtained from https://github.com/cybergis/COVID_19Accessibility

In [ ]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import time
from tqdm import tqdm, trange
from shapely.geometry import Point, MultiPoint
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings("ignore")

## Import Data

In [ ]:
# Supply: hospitals in the city of Chicago
hospitals = gpd.read_file('./data/Chicago_Hospital_Info.shp')
hospitals.head(1)

In [ ]:
# Demand: population per census tract
tracts = gpd.read_file('./data/Chicago_Tract.shp')
tracts.head(1)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

tracts.plot('TotalPop', ax=ax, scheme='FisherJenks', cmap='Blues')
hospitals.plot(markersize='Total_Bed', ax=ax, color='black')

In [ ]:
# Mobility: Chicago Road Network
G = ox.io.load_graphml('./data/chicago_road.graphml')
ox.plot_graph(G)

In [ ]:
'''
## This cell demonstrates how you can make the identical network dataset from OSMnx. 

# Obtain OSM Network in the city of Chicago
G = ox.graph_from_place('Chicago, IL, USA', network_type='drive', simplify=True)

# Remove edges
for u, v, data in G.copy().edges(data=True):
    if data['highway'] not in ['motorway', 'motorway_link', 
                               'trunk', 'trunk_link',
                               'primary', 'primary_link', 
                               'secondary', 'secondary_link']:
        G.remove_edge(u, v)
        
# Remove nodes
G.remove_nodes_from(list(nx.isolates(G)))

# Save OSM Network to Graphml
ox.io.save_graphml(G, './data/chicago_road.graphml')

'''

## Data Preprocessing

In [ ]:
# This function helps you to find the nearest OSM node from a given GeoDataFrame
# If geom type is point, it will take it without modification, but 
# IF geom type is polygon or multipolygon, it will take its centroid to calculate the nearest element. 

def find_nearest_osm(network, gdf):
    for idx, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
        if row.geometry.geom_type == 'Point':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                                    X=row.geometry.x, 
                                                    Y=row.geometry.y
                                                   )
        elif row.geometry.geom_type == 'Polygon' or row.geometry.geom_type == 'MultiPolygon':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                        X=row.geometry.centroid.x, 
                                        Y=row.geometry.centroid.y
                                       )
        else:
            print(row.geometry.geom_type)
            continue

        gdf.at[idx, 'nearest_osm'] = nearest_osm

    return gdf

supply = find_nearest_osm(G, hospitals)
demand = find_nearest_osm(G, tracts)

In [ ]:
# Extract the nodes and edges of the network dataset for the future analysis. 
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True)
nodes.head(1)

## Procedural demonstration

The following procedure demonstrates how to make a catchment area from a given location. 
1. Calculate accessible nodes in the network dataset from a given location with `nx.single_source_dijkstra_path_length()`. The function returns a dictionary (keys: OSMID, values: distance from a given location).  
2. Extract the locations (or coordinates) of accessible nodes based on the OSMID. 
3. Create a convex hull with the locations of the nodes. 

In [ ]:
# Threshold travel time
dist = 10000

# Calculate the number of census tract that is accessible to each hospital
# This returns the dictionary that has OSM node as its key and the distance as value.
temp_nodes = nx.single_source_dijkstra_path_length(G, supply.loc[0, 'nearest_osm'], dist, weight='length')
temp_nodes

In [ ]:
# Slice the `nodes` GeoDataFrame to collect the location of the nodes based on their OSMID. 
access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
access_nodes

In [ ]:
access_nodes.unary_union  # Union every nodes
access_nodes.unary_union.convex_hull # Create convex hull from the unioned nodes

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

tracts.plot('TotalPop', ax=ax, scheme='FisherJenks', cmap='Blues')

access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")
access_nodes.boundary.plot(ax=ax, color='red', linewidth=4)

supply_0 = supply.loc[supply.index==0]
supply_0.plot(markersize='Total_Bed', ax=ax, color='black')

In [ ]:
# In summary, the following is the necessary code to create a catchment area from a given location. 
dist = 10000

# 1. Calculate accessible nodes in the network dataset from a given location 
temp_nodes = nx.single_source_dijkstra_path_length(G, supply.loc[0, 'nearest_osm'], dist, weight='length')

# 2. Extract the locations (or coordinates) of accessible nodes based on the OSMID.
access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']

# 3. Create a convex hull with the locations of the nodes.
access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")

# Result. 
demand.loc[demand['geometry'].centroid.within(access_nodes[0])]

---
### *Exercise*

You may have understood the concept of 2SFCA method and how to calculate the catchment area. Here, we want to calculate the number of demand locations (i.e., census tract) accessible from each supply location (i.e., hospitals). This will be the preliminary supply-to-demand ratio of each hospital, assuming that there are **no weights** on the supply and demand. Then, return the result as `count` column of `supply_` GeoDataFrame (which is the copy of `supply` GeoDataFrame). <br>

The result should look like the below. 
![](./data/fig2.jpg)


```python
supply_ = supply.copy(deep=True)
supply_['count'] = 0

for i in range(supply.shape[0]):
    ### Your code here

    

    # Enter the number of demand location into the `count` column. 
    supply_.at[i, 'count'] = demand_count

```
Once we know the preliminary supply-to-demand ratio, we can incorporate the weigths of hospitals and census tracts, based on the bed count and population, respectively. 

---

In [ ]:
# Your code here

supply_ = supply.copy(deep=True)
supply_['count'] = 0

for i in range(supply.shape[0]):
    ### Your code here

    
    
    supply_.at[i, 'count'] = demand_count
    
supply_.head(5)

In [ ]:
""" Test code for the previous code. This cell should NOT give any errors when it is run."""
assert supply_.loc[6, 'count'] == 360
assert supply_.loc[24, 'count'] == 280

print('Success!')

## Measuring Accessibility to Hospitals in Chicago

With the skeleton we just created in the previous cell, we now want to incorporate the weights of supply and demand based on the bed count and population, respectively.

### Step1 of 2SFCA method: calculate the supply-to-demand ratio of each infrastructure (e.g., hospitals)

$$\large R_j = \frac{S_j}{\sum_{k\in {\left\{d_{kj} \le d_0 \right\}}}^{}P_k}$$
where<br>
$R_j$: the supply-to-demand ratio of location $j$. <br>
$S_j$: the degree of supply (e.g., number of doctors) at location $j$. <br>
$P_k$: the degree of demand (e.g., population) at location $k$. <br>
$d_{kj}$: the travel cost between locations $k$ and $j$. <br>
$d_0$: the threshold travel cost of the analysis. 


In [ ]:
supply_ = supply.copy(deep=True)
supply_['ratio'] = 0

for i in range(supply.shape[0]):
    
    # Create a catchment area from a given location
    temp_nodes = nx.single_source_dijkstra_path_length(G, supply.loc[i, 'nearest_osm'], dist, weight='length')
    access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
    access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")
    
    # Calculate the population within the catchment area
    temp_demand = demand.loc[demand['geometry'].centroid.within(access_nodes[0]), 'TotalPop'].sum()
    
    # Calculate the number of hospital beds in each hospital
    temp_supply = supply.loc[i, 'Total_Bed']
    
    # Calculate the number of hospital beds available for 100,000 people 
    supply_.at[i, 'ratio'] = temp_supply / temp_demand * 100000
    
supply_.head(5)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 10))

tracts.plot('TotalPop', ax=axes[0], scheme='FisherJenks', cmap='Blues')
supply_.plot(markersize=supply_['ratio'] * 5, ax=axes[0], color='black')
axes[0].set_title('Supply-to-demand ratio of each hospital')

tracts.plot('TotalPop', ax=axes[1], scheme='FisherJenks', cmap='Blues')
hospitals.plot(markersize='Total_Bed', ax=axes[1], color='black')
axes[1].set_title('Staffed hospital beds of each hospital')

plt.show()

### Step2 of 2SFCA method: sum the ratio from the locations of demand.

$$\large A_i = \sum_{j\in {\left\{d_{ij} \le d_0 \right\}}} R_j$$
where<br>
$A_i$: the accessibility measures at location $i$. <br>
$R_j$: the supply-to-demand ratio of location $j$. <br>
$d_{kj}$: the travel cost between locations $k$ and $j$. <br>
$d_0$: the threshold travel cost of the analysis. 


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

tracts.plot('TotalPop', ax=ax, scheme='FisherJenks', cmap='Blues')
tracts.loc[tracts.index==105].plot(ax=ax, color='red')

# Plot catchment area
temp_nodes = nx.single_source_dijkstra_path_length(G, demand.loc[105, 'nearest_osm'], dist, weight='length')
access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")
access_nodes.boundary.plot(ax=ax, color='red', linewidth=4)

# Calculate accessible hospital from a census tract
access_supply_ = supply_.loc[supply_['geometry'].centroid.within(access_nodes[0])]
non_access_supply_ = supply_.loc[~supply_['geometry'].centroid.within(access_nodes[0])]

access_supply_.plot(ax=ax, markersize=access_supply_['ratio']*5, color='red')
non_access_supply_.plot(ax=ax, markersize=non_access_supply_['ratio']*5, color='grey')

In [ ]:
demand_ = demand.copy(deep=True)
demand_['access'] = 0

for j in trange(demand.shape[0]):
    temp_nodes = nx.single_source_dijkstra_path_length(G, demand.loc[j, 'nearest_osm'], dist, weight='length')
    access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
    access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")
    
    accum_ratio = supply_.loc[supply_['geometry'].within(access_nodes[0]), 'ratio'].sum()
    demand_.at[j, 'access'] = accum_ratio
    
demand_

In [ ]:
# Plotting accessibility measurement result. 
fig, ax = plt.subplots(figsize=(10,10))

demand_.plot('access', ax=ax, figsize=(10,10), legend=True, cmap='Blues', scheme='FisherJenks')
demand_.boundary.plot(ax=ax, linestyle='dotted', lw=0.5, color='black', zorder=1)

---
### *Exercise*
It may be tedious if we need to write the two-step floating catchment area every single time when the study area or data is changed. Therefore, we want to make those two steps into two functions so that we can increase the replicability and reproducibility of the study. <br><br>
The following code was copied and pasted from the previous cells and describes the necessary procedures. Here, you need to change the name of the variable according to the inputs of each function. 

```python
def step1_2SFCA(supply, supply_attr, demand, demand_attr, mobility, threshold):
    """
    Input: 
    - supply (GeoDataFrame): stores locations and attributes of supply 
    - supply_attr (str): the column of `supply` to be used for the analysis
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - demand_attr (str): the column of `demand` to be used for the analysis
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - supply_ (GeoDataFrame): 
      a copy of supply and it stores supply-to-demand ratio of each supply at `ratio` column
    """
    
    # Your code here (Change the name of the variable according to the inputs)
    supply_ = supply.copy(deep=True)
    supply_['ratio'] = 0

    for i in range(supply.shape[0]):

        # Create a catchment area from a given location
        temp_nodes = nx.single_source_dijkstra_path_length(G, supply.loc[i, 'nearest_osm'], dist, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")

        # Calculate the population within the catchment area
        temp_demand = demand.loc[demand['geometry'].centroid.within(access_nodes[0]), 'TotalPop'].sum()

        # Calculate the number of hospital beds in each hospital
        temp_supply = supply.loc[i, 'Total_Bed']

        # Calculate the number of hospital beds available for 100,000 people 
        supply_.at[i, 'ratio'] = temp_supply / temp_demand * 100000

    
    return supply_


def step2_2SFCA(result_step1, demand, mobility, threshold):
    """
    Input: 
    - result_step1 (GeoDataFrame): stores locations and 'ratio' attribute that resulted in step1 
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - demand_ (GeoDataFrame): 
      a copy of demand and it stores the final accessibility measures of each demand location at `ratio` column
    """
    
    # Your code here (Change the name of the variable according to the inputs)
    demand_ = demand.copy(deep=True)
    demand_['access'] = 0

    for j in trange(demand.shape[0]):
        temp_nodes = nx.single_source_dijkstra_path_length(G, demand.loc[j, 'nearest_osm'], dist, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")

        accum_ratio = supply_.loc[supply_['geometry'].within(access_nodes[0]), 'ratio'].sum()
        demand_.at[j, 'access'] = accum_ratio

    return demand_
```

---

In [ ]:
def step1_2SFCA(supply, supply_attr, demand, demand_attr, mobility, threshold):
    """
    Input: 
    - supply (GeoDataFrame): stores locations and attributes of supply 
    - supply_attr (str): the column of `supply` to be used for the analysis
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - demand_attr (str): the column of `demand` to be used for the analysis
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - supply_ (GeoDataFrame): 
      a copy of supply and it stores supply-to-demand ratio of each supply at `ratio` column
    """
    
    # Your code here (Change the name of the variable according to the inputs)
    supply_ = supply.copy(deep=True)
    supply_['ratio'] = 0

    for i in range(supply.shape[0]):

        # Create a catchment area from a given location
        temp_nodes = nx.single_source_dijkstra_path_length(G, supply.loc[i, 'nearest_osm'], dist, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")

        # Calculate the population within the catchment area
        temp_demand = demand.loc[demand['geometry'].centroid.within(access_nodes[0]), 'TotalPop'].sum()

        # Calculate the number of hospital beds in each hospital
        temp_supply = supply.loc[i, 'Total_Bed']

        # Calculate the number of hospital beds available for 100,000 people 
        supply_.at[i, 'ratio'] = temp_supply / temp_demand * 100000

    
    return supply_


def step2_2SFCA(result_step1, demand, mobility, threshold):
    """
    Input: 
    - result_step1 (GeoDataFrame): stores locations and 'ratio' attribute that resulted in step1 
    - demand (GeoDataFrame): stores locations and attributes of demand 
    - mobility (NetworkX MultiDiGraph): Network Dataset obtained from OSMnx
    - threshold (int): threshold travel distance
    
    Output:
    - demand_ (GeoDataFrame): 
      a copy of demand and it stores the final accessibility measures of each demand location at `ratio` column
    """
    
    # Your code here (Change the name of the variable according to the inputs)
    demand_ = demand.copy(deep=True)
    demand_['access'] = 0

    for j in trange(demand.shape[0]):
        temp_nodes = nx.single_source_dijkstra_path_length(G, demand.loc[j, 'nearest_osm'], dist, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        access_nodes = gpd.GeoSeries(access_nodes.unary_union.convex_hull, crs="EPSG:4326")

        accum_ratio = supply_.loc[supply_['geometry'].within(access_nodes[0]), 'ratio'].sum()
        demand_.at[j, 'access'] = accum_ratio

    return demand_

In [ ]:
# Check your answer here
# This will create a map with the accessibility measures with 5000-meter threshold. 
dist = 5000

step1 = step1_2SFCA(supply, 'Total_Bed', demand, 'TotalPop', G, dist)
step2 = step2_2SFCA(step1, demand, G, dist)

# Plotting accessibility measurement result. 
fig, ax = plt.subplots(figsize=(10,10))

step2.plot('access', ax=ax, figsize=(10,10), legend=True, cmap='Blues', scheme='FisherJenks')
step2.boundary.plot(ax=ax, linestyle='dotted', lw=0.5, color='black', zorder=1)

## Comparison with Social Vulnerability Index (SVI)

Social vulnerability refers to the potential negative effects on communities caused by external stresses on human health. Such stresses include natural or human-caused disasters, or disease outbreaks. Reducing social vulnerability can decrease both human suffering and economic loss. <br>
The CDC/ATSDR Social Vulnerability Index (CDC/ATSDR SVI) uses 15 U.S. census variables to help local officials identify communities that may need support before, during, or after disasters.

Social Velnerabiltiy Index provides the following data. Percentile ranking values range **from 0 to 1, with higher values indicating greater vulnerability**.
- RPL_THEME1: Socioeconomic Status
- RPL_THEME2: Household Composition & Disability
- RPL_THEME3: Minority Status & Language
- RPL_THEME4: Housing Type & Transportation

![](./data/fig3.jpg)


Source: https://www.atsdr.cdc.gov/placeandhealth/svi/index.html

In [ ]:
svi = gpd.read_file('./data/SVI2018_ILLINOIS_tract.shp')
svi = svi[['FIPS','RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES', 'geometry']]
svi = svi.to_crs(epsg=4326)
svi = svi.loc[svi['FIPS'].isin(tracts['GEOID'].tolist())]
svi = svi.replace(-999, 0)
svi = svi.reset_index(drop=True)
svi.head()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 10))

for idx, ax in enumerate(axes):
    svi.plot(f'RPL_THEME{idx+1}', ax=ax, cmap='Reds', scheme='FisherJenks', legend=True)
    ax.get_xaxis().set_visible(False)  # Remove ticks and labels
    ax.get_yaxis().set_visible(False)  # Remove ticks and labels
    ax.set_title(f'RPL_THEME{idx+1}')

In [ ]:
# Perason's relation between accessibility and SVI
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

ax = axes.reshape(-1)
for idx in range(0, 4, 1):
    
    ax[idx].scatter(step2['access'], svi[f'RPL_THEME{idx+1}'])
    ax[idx].set_title(f'RPL_THEME{idx+1}')
#     print(idx+1, pearsonr(step2['access'], svi[f'RPL_THEME{idx+1}']))